Load libraries

In [1]:
import gc
import time
import numpy as np
import pandas as pd
import time
from typing import List, Dict, Set
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 修改当前文件夹位置 假定notebook文件就在项目文件夹根目录
import os
def get_root_dir():
    if os.path.exists('/content/drive/MyDrive/'):
        return '/content/drive/MyDrive/Colab/4-AMEX/AMEX Project/notebooks' #在Colab里
    else:
        return './' #在本地

#调用系统命令，相当于cd，但是直接!cd是不行的
os.chdir(get_root_dir())

Load data

In [4]:
df = pd.read_parquet('../data/4-PreCompressed/FilledWithRandomForest/full_filled.parquet', engine = 'pyarrow')
print(df.shape)
df.head(2)

(16895213, 173)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,-1,0,0,0.0,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,-1,0,0,0.0,0,0.005492,0


##Part I: Create interaction variables

In [5]:
def create_interaction(df: pd.DataFrame, cat_cols: List[str], cont_cols: List[str]) -> pd.DataFrame:
  # dict for holding created variables
  features = {}
  card_all = df['customer_ID'].values.tolist()
  for card in card_all:
      features[card] = {}
  print("Starting ...")
  # create interaction variables
  num = 0
  t = time.time()
  for i, row in df.iterrows():
    card = row['customer_ID']
    for cat_col in cat_cols:
        for cont_col in cont_cols:
            col_name = '&'.join([cat_col, str(row[cat_col]), cont_col])
            features[card][col_name] = features[card].get(col_name, 0) + row[cont_col]
    num += 1
    if num % 100000 == 0:
      dur = round((time.time() - t) / 60, 2)
      print(f'{num} records processed in {dur} min.')

  # convert dict to df
  df = pd.DataFrame(features).T.reset_index()
  df.fillna(0, inplace=True)
  cols = df.columns.tolist()
  df.columns = ['customer_ID'] + cols[1:]
  return df

In [6]:
with open('../data/cont_ids.json', 'r') as f:
  cont_cols_l = json.load(f)
print(len(cont_cols_l))

8


In [7]:
cat_cols = ['B_30', 'B_38', 'D_64', 'D_68', 'D_114', 'D_117', 'D_120']

In [8]:
cont_cols = cont_cols_l[6]
half = len(cont_cols) // 2
cont_cols1, cont_cols2 = cont_cols[:half], cont_cols[half:]
cols1, cols2 = ['customer_ID'] + cat_cols + cont_cols1, ['customer_ID'] + cat_cols + cont_cols2
df1, df2 = df[cols1], df[cols2]
print(df1.shape, df2.shape)
del df
gc.collect()

(16895213, 18) (16895213, 18)


66

In [ ]:
df1 = create_interaction(df1, cat_cols, cont_cols1)
df1.to_parquet('../data/6-interaction/full_7_1.parquet', index=False)
print(df1.shape)

Starting ...
100000 records processed in 1.1 min.
200000 records processed in 2.08 min.
300000 records processed in 3.07 min.
400000 records processed in 4.06 min.
500000 records processed in 5.05 min.
600000 records processed in 6.04 min.
700000 records processed in 7.03 min.
800000 records processed in 8.04 min.
900000 records processed in 9.04 min.
1000000 records processed in 10.03 min.
1100000 records processed in 11.02 min.
1200000 records processed in 12.01 min.
1300000 records processed in 13.01 min.
1400000 records processed in 14.0 min.
1500000 records processed in 14.99 min.
1600000 records processed in 15.98 min.
1700000 records processed in 16.97 min.
1800000 records processed in 17.97 min.
1900000 records processed in 18.96 min.
2000000 records processed in 19.95 min.
2100000 records processed in 20.94 min.
2200000 records processed in 21.93 min.
2300000 records processed in 22.92 min.
2400000 records processed in 23.9 min.
2500000 records processed in 24.89 min.
2600000 

In [ ]:
del df1
gc.collect()

22

In [ ]:
df2 = create_interaction(df2, cat_cols, cont_cols2)
df2.to_parquet('../data/6-interaction/full_7_2.parquet', index=False)
print(df2.shape)

Starting ...
100000 records processed in 1.14 min.
200000 records processed in 2.13 min.
300000 records processed in 3.12 min.
400000 records processed in 4.11 min.
500000 records processed in 5.09 min.
600000 records processed in 6.09 min.
700000 records processed in 7.08 min.
800000 records processed in 8.07 min.
900000 records processed in 9.28 min.
1000000 records processed in 10.28 min.
1100000 records processed in 11.28 min.
1200000 records processed in 12.28 min.
1300000 records processed in 13.28 min.
1400000 records processed in 14.27 min.
1500000 records processed in 15.27 min.
1600000 records processed in 16.28 min.
1700000 records processed in 17.27 min.
1800000 records processed in 18.28 min.
1900000 records processed in 19.33 min.
2000000 records processed in 20.33 min.
2100000 records processed in 21.32 min.
2200000 records processed in 22.32 min.
2300000 records processed in 23.32 min.
2400000 records processed in 24.32 min.
2500000 records processed in 25.31 min.
26000

## Part II: Compress continuous features and join with interaction variables

In [ ]:
def compress_features(df: pd.DataFrame, cat_cols: List[str], cont_cols: List[str]) -> pd.DataFrame:
  df = df.copy()
  cid = pd.Categorical(df.pop('customer_ID'), ordered=True) 
  last = (cid != np.roll(cid, -1)) # mask for last statement of every customer

  # compress continuous features: mean, min, max, last
  cont_df = df[['customer_ID'] + cont_cols]
  cont_avg = (cont_df
              .groupby('customer_ID')
              .mean()
              .rename(columns={f: f"{f}_avg" for f in cont_cols})
             )
  print('Avg has been computed')
  gc.collect()
  
  cont_max = (cont_df
            .groupby('customer_ID')
            .max()
            .rename(columns={f: f"{f}_max" for f in cont_cols})
            )
  print('Max has been computed')
  gc.collect()
  
  cont_min = (cont_df
            .groupby('customer_ID')
            .min()
            .rename(columns={f: f"{f}_min" for f in cont_cols})
            )
  print('Min has been computed')
  gc.collect()
  
  cont_last = (cont_df.loc[last, :]
              .rename(columns={f: f"{f}_last" for f in cont_cols})
              .set_index(np.asarray(cid[last]))
            )
  df = None # we no longer need the original data
  print('Last has been computed for ', i)


In [ ]:
['customer_ID'] + ['a', 'b', 'c']

['customer_ID', 'a', 'b', 'c']